In [205]:
import csv
import math
movie_filename = "/Users/sam.boyarsky/data/movie_lens/ml-100k/u.item"
rating_filename = "/Users/sam.boyarsky/data/movie_lens/ml-100k/u.data"
user_filename = "/Users/sam.boyarsky/data/movie_lens/ml-100k/u.user"

In [215]:
class Movie:
    def __init__(self, **kwargs):
        self.movie_id = int(kwargs["movie_id"])
        self.title = kwargs["title"]
    
    def __repr__(self):
        return "{} {}".format(self.movie_id, self.title)
    
    def get_avg(self, rating_dict):
        
        return 6
    
class User:
    def __init__(self, **kwargs):
        self.user_id = int(kwargs["user_id"])
        self.age = kwargs["age"]
        self.gender = kwargs["gender"]
        self.occupation = kwargs["occupation"]

    def __repr__(self):
        return "{} {} {} {}".format(self.user_id, self.occupation, self.age, self.gender)

class Rating:
    def __init__(self, **kwargs):
        self.user_id = int(kwargs["user_id"])
        self.movie_id = int(kwargs["movie_id"])
        self.rating = float(kwargs["rating"])
    
    def __repr__(self):
        return "{} {} {}".format(self.user_id, self.movie_id, self.rating)

In [216]:
!head -5 /Users/sam.boyarsky/data/movie_lens/ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0


In [217]:
def load_movies(movie_filename):
    movie_dict = {}
    with open(movie_filename, encoding="latin-1") as f:
        headers = ["movie_id", "title"]
        reader = csv.DictReader(f, fieldnames=headers, delimiter="|")
        for row in reader:
            del row[None]
            m = Movie(**row)
            movie_dict[m.movie_id] = m
    return movie_dict


def load_users(user_filename):
    user_dict = {}
    with open(user_filename, encoding="latin-1") as f:
        headers = ["user_id", "age", "gender", "occupation"]
        reader = csv.DictReader(f, fieldnames=headers, delimiter="|")
        for row in reader:
            del row[None]
            u = User(**row)
            user_dict[u.user_id] = u
    return user_dict


def load_ratings(rating_filename):
    ratings_by_movie_id = {}
    ratings_by_user_id = {}
    with open(rating_filename, encoding="latin-1") as f:
        headers = ["user_id", "movie_id", "rating"]
        reader = csv.DictReader(f, fieldnames=headers, delimiter="\t")
        for row in reader:
            del row[None]
            r = Rating(**row)
            ratings_by_movie_id.setdefault(r.movie_id, []).append(r)
            ratings_by_user_id.setdefault(r.user_id, []).append(r)
    return ratings_by_movie_id, ratings_by_user_id

In [218]:
md = load_movies(movie_filename)
ud = load_users(user_filename)
rdbm, rdbu = load_ratings(rating_filename)

In [219]:
def avg_rating_by_movie_id(movie_id):
    list_of_ratings = rdbm[movie_id]
    avg = sum([r.rating for r in list_of_ratings])/len(list_of_ratings)
#    print(md[movie_id], avg)
    return avg

In [220]:
def top_x_movies(x, min_ratings=5):
    avg_ratings = {}
    for movie in md.values():
        if len(rdbm[movie.movie_id]) < min_ratings: continue
        avg = avg_rating_by_movie_id(movie.movie_id)
        avg_ratings[movie.title] = avg
    count = 0
    for title, avg_rating in sorted(avg_ratings.items(), key=lambda x: x[1], reverse=True):
        if count >= x:
            return
        count += 1
        print(count, title, avg_rating)

In [221]:
#top_x_movies(20,4)

In [222]:
def euclidean_distance(v, w):
    """Given two lists, give the Euclidean distance between them on a scale
    of 0 to 1. 1 means the two lists are identical.
    """

    # Guard against empty lists.
    if len(v) is 0:
        return 0

    # Note that this is the same as vector subtraction.
    differences = [v[idx] - w[idx] for idx in range(len(v))]
    squares = [diff ** 2 for diff in differences]
    sum_of_squares = sum(squares)

    return 1 / (1 + math.sqrt(sum_of_squares))

In [223]:
def similarity(u1, u2):
#    same_list = []
    u1_list = []
    u2_list = []
    for u1_rating in rdbu[u1]:
        for u2_rating in rdbu[u2]:
            if u1_rating.movie_id == u2_rating.movie_id:
                u1_list.append(u1_rating)
                u2_list.append(u2_rating)
                #same_list.append((u1_rating, u2_rating))
    return euclidean_distance([r.rating for r in u1_list], [r.rating for r in u2_list])
#    return euclidean_distance([r[0].rating for r in same_list],
#                            [r[1].rating for r in same_list])
    

In [224]:
similarity(1,4)

0.18660549686337075

In [225]:
md[1].get_avg()

6